# xapm_db_os_stat, xapm_db_stat 데이터 분석

by pdm

In [17]:
from __future__ import division
import google.datalab.bigquery as bq
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd

# BigQuery로 xapm_db 테이블 불러오기

In [11]:
%bq tables list --project exem-191100 --dataset ods_im5

In [22]:
%%bq query --name osstat
SELECT *, (total_memory - free_memory)/total_memory as mem_use_rate
FROM `exem-191100.ods_im5.xapm_db_os_stat`
ORDER BY db_id, time 
LIMIT 10

In [23]:
%%bq sample --query osstat --count 10

cpu_usage,db_id,free_memory,max_cpu_usage,min_free_memory,time,total_memory,mem_use_rate
5244,6,246592,6670,241860,2017-11-03 15:00:00,6117496,0.959690696978
5201,6,246007,5720,241844,2017-11-03 15:01:00,6117496,0.959786324339
5168,6,245579,5460,241372,2017-11-03 15:02:00,6117496,0.959856287605
5157,6,245377,5330,241372,2017-11-03 15:03:00,6117496,0.959889307651
5263,6,242621,5610,240992,2017-11-03 15:04:00,6117496,0.960339818776
5220,6,244343,5600,240256,2017-11-03 15:05:00,6117496,0.960058331056
5218,6,243530,5530,239876,2017-11-03 15:06:00,6117496,0.960191228568
5274,6,243200,5630,239636,2017-11-03 15:07:00,6117496,0.960245172208
5142,6,242762,5430,239132,2017-11-03 15:08:00,6117496,0.960316770129
5260,6,242303,5520,238636,2017-11-03 15:09:00,6117496,0.960391800828


In [1]:
%%bq query --name grouping
SELECT db_id, COUNT(db_id) as count, MAX(mem_use_rate) as max_mem_use_rate, AVG(mem_use_rate) as avg_mem_use_rate, MIN(mem_use_rate) as min_mem_use_rate
FROM(
  SELECT db_id, (total_memory - free_memory)/total_memory as mem_use_rate
  FROM `exem-191100.ods_im5.xapm_db_os_stat`
  )
GROUP BY db_id

In [2]:
%%bq sample --query grouping

db_id,count,max_mem_use_rate,avg_mem_use_rate,min_mem_use_rate
16,44301,0.993507155516,0.987811373303,0.866980441022
6,44306,0.979446655952,0.953953761297,0.898364625003
13,42378,0.971808579389,0.96437537027,0.936422909007


xapm_db_os_stat db에는 **{ 6, 13, 16 } 3개의 db**에 대한 데이터가 있으며 그 **갯수는 약 40000개로 균일**하고, **16번 db가 가장 많은 메모리 점유율**을 갖고 있음을 알 수 있다.

In [1]:
%%bq query --name osstat_time_check
SELECT *, (total_memory - free_memory)/total_memory as mem_use_rate
FROM `exem-191100.ods_im5.xapm_db_os_stat`
WHERE db_id = 6
ORDER BY time DESC

In [2]:
%%bq sample --query osstat_time_check

cpu_usage,db_id,free_memory,max_cpu_usage,min_free_memory,time,total_memory,mem_use_rate
5257,6,288554,5690,285708,2017-12-04 11:19:00,6117496,0.952831354528
5138,6,289245,5430,285584,2017-12-04 11:18:00,6117496,0.952718399816
5164,6,289808,5410,286212,2017-12-04 11:17:00,6117496,0.952626368697
5177,6,289573,5470,285716,2017-12-04 11:16:00,6117496,0.952664783107
5273,6,290510,5590,286956,2017-12-04 11:15:00,6117496,0.952511615864
5391,6,290252,5860,286336,2017-12-04 11:14:00,6117496,0.952553789982
5360,6,290828,5720,287420,2017-12-04 11:13:00,6117496,0.952459633811
5282,6,291217,5600,287692,2017-12-04 11:12:00,6117496,0.952396045702
5270,6,291617,5730,287932,2017-12-04 11:11:00,6117496,0.952330659472
5340,6,290821,5840,288064,2017-12-04 11:10:00,6117496,0.95246077807


# 막 지어낸 anomaly detection 방법1

방법
- avg_mem_use_rate가 특정 threshold 이상, 예를들면 0.99 이상 이면 시스템이 위험한 상태라고 판단하고, 해당 상태의 timestamp의 db를 anomaly라고 할 수 있다.

한계
- 하지만 모든 메모리의 평균 점유율이 0.95 이상으로 상당히 높으며, cache, swap 중인 메모리의 정보가 없으므로 유의미 하지는 않을거라 생각된다.
- 이것 보다는, 어떤 작업을 처리하기 위한 db_time 정보가 필요해 보인다

궁금한점
- cpu_usage 가 높으면 시스템에 해가 되는 것인가..?

# 막 지어낸 anomaly detection 방법2

- 중간에 빠진데이터가 있던데 그게 anomaly가 아닐까?